In [1]:
import math
import random
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np
import numba
import numpy.random
from collections import deque

import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam

In [2]:
class GridWorld:
    def __init__(self, taille=[4, 4], position_start=[0, 1], good_end_position=[3, 2], bad_end_position=[0, 3]):
        self.current_state = position_start  # État actuel (ligne, colonne)
        self.states = [[x, y] for x in range(taille[0]) for y in range(taille[1])]
        self.end_good_state = good_end_position  # État final (ligne, colonne)
        self.end_bad_state = bad_end_position
        self.grid_size = taille  # Taille de la grille (lignes, colonnes)
        self.stateSpace = {}
        self.matchStates()
        self.currentIntState = self.getStateInt(self.current_state)
        self.num_actions = 4  # Nombre total d'actions possibles (haut, bas, gauche, droite)
        self.reward = 0  # Récompense actuelle
        self.done = False  # Indique si la partie est terminée
        self.generate_grid()
        self.actions = [0, 1, 2, 3]
        self.rewards = [0, 1, 3]
        self.actionSpace = {0: -self.grid_size[0], 1: self.grid_size[0],
                            2: -1, 3: 1}

    def state_description(self):
        return np.array([self.currentIntState / (len(self.states) - 1) * 2.0 - 1.0])

    def state_dim(self):
        return len(self.state_description())

    def reset(self):
        self.done = False
        self.current_state = [np.random.randint(0, self.grid_size[0] - 1), np.random.randint(0, self.grid_size[1] - 1)]
        self.currentIntState = self.getStateInt(self.current_state)
        self.reward = 0

    def matchStates(self):
        i = 0
        for s in self.states:
            self.stateSpace[str(s)] = i
            i = i + 1

    def getStateInt(self, st):
        return self.stateSpace[str(st)]

    def getStateCouple(self, st):
        n_state = {i for i in self.stateSpace if self.stateSpace[i] == st}
        return list(n_state)

    def step(self, action):
        if action == 0:
            if self.current_state[0] == 0:
                self.current_state[0] = self.grid_size[0] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour traverser le mur
                #self.generate_grid()
                self.endgame()
            else:
                self.current_state[0] = self.current_state[0] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()

        elif action == 1:
            if self.current_state[0] == self.grid_size[0] - 1:
                self.current_state[0] = 0
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                self.generate_grid()
                self.endgame()
            else:
                self.current_state[0] = self.current_state[0] + 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()

        elif action == 2:
            if self.current_state[1] == 0:
                self.current_state[1] = self.grid_size[1] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
            else:
                self.current_state[1] = self.current_state[1] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()

        elif action == 3:
            if self.current_state[1] == self.grid_size[1] - 1:
                self.current_state[1] = 0
                self.currentIntState = self.getStateInt(self.current_state)
                # print(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
            else:
                self.current_state[1] = self.current_state[1] + 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = 0  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
                # Si l'on atteint l'état final, la partie est terminée
        return self.currentIntState, self.reward, self.done

    def endgame(self):
        if self.current_state == self.end_good_state:
            self.reward = 10  # Récompense de 10 pour atteindre l'état final
            self.done = True
            print("Une bonne récompense")
        elif self.current_state == self.end_bad_state:
            self.reward = -10
            self.done = True
            print("Une mauvaise récompense")

    def generate_grid(self):
        grid = []
        for i in range(self.grid_size[0]):
            grid.append([])
            for j in range(self.grid_size[1]):
                grid[i].append("_")
        grid[self.current_state[0]][self.current_state[1]] = "X"
        for i in grid:
            print(i)
        print("\n")

In [3]:
def build_compile_model(env):
    model = tf.keras.models.Sequential()
    model.add(Dense(24, input_dim=1, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(len(env.actions), activation='linear'))

    model.compile(loss='mse', optimizer=Adam(learning_rate=0.01))
    return model

In [4]:
def ddqn_er(env, max_iter=1000, gamma=0.99, alpha=0.1, epsilon=0.1):
    q_network = build_compile_model(env)
    target_network = build_compile_model(env)
    nb_steps = 0
    first_episode = True
    
    cumumated_reward = 0
    step = 0
    reward = 0
    reward_per_episode = []
    steps_by_episode = ()
    step_by_episode = []
    batch_size = 32
    memory = deque(maxlen=2000)
    
    for iteretion in range(max_iter):
        print("Iteration : ", iteretion)
        if env.done:
            reward_per_episode.append(env.reward)
            step_by_episode.append(step)
            env.reset()
            step = 0
            cumumated_reward = 0

        current_state = env.currentIntState
        actions = env.actions
        q_values = q_network.predict(np.array([current_state]))[0]
        if np.random.rand() < epsilon:
            a = np.random.choice(actions)
        else:
            a= np.argmax(q_values)

        old_reward = env.reward
        new_state, reward, done = env.step(a)
        #new_state = env.state_description()
        memory.append((old_reward, a, reward, new_state, done))
            
        if (len(memory) > batch_size):
            minibatch = random.sample(memory, batch_size)
            for s, ac, rreward, ns, terminated in minibatch:
                q_values = q_network.predict(np.array([s]))[0]
                if done:
                    q_values[a] = rreward
                else:
                    t = target_network.predict(np.array([new_state]))[0]
                q_values[a] = rreward + gamma * np.amax(t)
        q_network.fit(np.array([current_state]), np.array([q_values]), verbose=0)
        
        cumumated_reward += reward
        step += 1

        if done:
            target_network.set_weights(q_network.get_weights())
                
    return reward_per_episode, step_by_episode

In [ ]:
world = GridWorld()
scores, steps = ddqn_er(world, max_iter = 1000)
plt.plot(scores)
plt.show()
plt.plot(steps)
plt.show()

['_', 'X', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


Iteration :  0
1/1 [==============================] - 0s 188ms/step
Iteration :  1
1/1 [==============================] - 0s 42ms/step
Iteration :  2
1/1 [==============================] - 0s 44ms/step
Iteration :  3
1/1 [==============================] - 0s 46ms/step
Iteration :  4
1/1 [==============================] - 0s 54ms/step
Iteration :  5
1/1 [==============================] - 0s 49ms/step
Une bonne récompense
Iteration :  6
1/1 [==============================] - 0s 47ms/step
Iteration :  7
1/1 [==============================] - 0s 42ms/step
Iteration :  8
1/1 [==============================] - 0s 40ms/step
Iteration :  9
1/1 [==============================] - 0s 54ms/step
Iteration :  10
1/1 [==============================] - 0s 34ms/step
Iteration :  11
1/1 [==============================] - 0s 34ms/step
Iteration :  12
1/1 [==============================] - 0s 43ms/step
Iteration :  13
1/

1/1 [==============================] - 0s 35ms/step
Iteration :  34
1/1 [==============================] - 0s 38ms/step
Iteration :  35
1/1 [==============================] - 0s 35ms/step
Iteration :  36
1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 38ms/step
Iteration :  37
1/1 [==============================] - 0s 39ms/step
Iteration :  38
1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 38ms/step
Iteration :  39
1/1 [==============================] - 0s 38ms/step
Iteration :  40
1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 32ms/step
Iteration :  41
1/1 [==============================] - 0s 39ms/step
Iteration :  42
1/1 [==============================] - 0s 37ms/step
Une mauvaise récompense
1/1 [==============================] - 0s 35ms/step
Iteration :  43
1/1 [==============================] - 0s 30ms/step


...

1/1 [==============================] - 0s 39ms/step
Iteration :  56
1/1 [==============================] - 0s 43ms/step
Iteration :  57
1/1 [==============================] - 0s 33ms/step
Iteration :  58
1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 35ms/step
Iteration :  59
1/1 [==============================] - 0s 31ms/step
Iteration :  60
1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 37ms/step
Iteration :  61
1/1 [==============================] - 0s 35ms/step
Iteration :  62
1/1 [==============================] - 0s 34ms/step
Iteration :  63
1/1 [==============================] - 0s 36ms/step


1/1 [==============================] - 0s 59ms/step
Iteration :  64
1/1 [==============================] - 0s 37ms/step
Iteration :  65
1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 57ms/step
Iteration :  66
1/1 [==============================] - 0s 33ms/step
Iteration :  67
1/1 [==============================] - 0s 34ms/step
